# MQTT 

In this notebook we will use PAHO as our MQTT broker.

- https://pypi.org/project/paho-mqtt/
- 

## 0. Install packages

In [1]:
!pip install paho-mqtt

## 1. One sccript for sending and receiving using iot.eclipse.org as broker

In [2]:
# source: http://www.steves-internet-guide.com/python-mqtt-publish-subscribe/
# source: http://www.steves-internet-guide.com/mqtt-username-password-example/ => if you want to use eclipse you need account+pw

import time
import paho.mqtt.client as paho
broker="broker.hivemq.com"
broker="iot.eclipse.org"
#define callback
def on_message(client, userdata, message):
    time.sleep(1)
    print("received message =",str(message.payload.decode("utf-8")))

client= paho.Client("client-001") #create client object client1.on_publish = on_publish #assign function to callback client1.connect(broker,port) #establish connection client1.publish("house/bulb1","on")
######Bind function to callback
client.on_message=on_message
#####
print("connecting to broker ",broker)
client.connect(broker)#connect
client.loop_start() #start loop to process received messages
print("subscribing ")
client.subscribe("house/bulb1")#subscribe
time.sleep(2)
print("publishing ")
client.publish("house/bulb1","on")#publish
time.sleep(4)
client.disconnect() #disconnect
client.loop_stop() #stop loop


connecting to broker  iot.eclipse.org


TimeoutError: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond